# AdaBoost

AdaBoost (adaptive boosting) 是一种集成算法，可以通过集成多个弱分类器，例如单层决策树，实现较高分类性能的效果。

## AdaBoost的一般流程

1. 收集数据：任意方法
2. 准备数据：依赖于所使用的弱分类器类型。
3. 分析数据：任意方法
4. 训练算法：time-consuming，分类器将多次在同一数据集上训练弱分类器
5. 测试算法：计算分类的错误类
6. 使用算法：同SVM一样，AdaBoost也是二元分类器。多个类别的判别需要拓展

AdaBoost的工作流程如下：
- 训练数据中的每个样本，并赋予其一个权重，这些权重构成了向量 ${D}$ . 这些权重具有相同的初始值
- 在训练数据上训练出一个弱分类器并计算该分类器的错误率
- 调整样本权重值。较低正确分类样本权重而提高错分样本权重，再训练第二个弱分类器。直到用户指定的弱分类器数目，或分类器错误率为0

AdaBoost为每个分类器分配了一个权重值alpha，该alpha值基于每个弱分类器的错误率计算。其中，

错误率：${e=\frac{mis-classification}{all samples}}$ 

alpha: ${\alpha = 0.5*ln(\frac{1-e}{e})}$

计算出alpha后，对权重向量 ${D}$ 进行更新。

如果样本被正确分类，其权重更改为：

${D^{t+1}_i = \frac{D^{t}_i e^{-\alpha }}{Sum(D)}}$

如果样本被错分，其权重更改为：

${D^{t+1}_i = \frac{D^{t}_i e^{\alpha }}{Sum(D)}}$


接下来，根据书中的例子构建基于单层决策树的AdaBoost。

首先，构建单层决策树（decision stump）的弱分类器

构建数据与标签
```python
def loadSimpData():
    datMat = matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels
```


In [1]:
import adaboost
datMat, classLabels = adaboost.loadSimpData()
datMat

matrix([[1. , 2.1],
        [2. , 1.1],
        [1.3, 1. ],
        [1. , 1. ],
        [2. , 1. ]])

In [2]:
classLabels

[1.0, 1.0, -1.0, -1.0, 1.0]

为了找出最佳的单层决策树，我们需要：

遍历数据集的每个特征；

    对每个特征根据步长遍历特征值；
    
        对每个步长，遍历两种不等号情况（大于或小于）
        
        建立单层决策树并利用加权数据集进行测试
        
        如果错误率低于minError，则返回当前单层决策树为最佳单层决策树
        
返回单层决策树


In [5]:
from numpy import *
D = mat(ones( (5,1) )/5)
adaboost.buildStump(dataArr=datMat, classLabels=classLabels, D=D)

({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'},
 matrix([[0.2]]),
 array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

接下来，实现完整的AdaBoost：

对每次迭代：

    利用 `buildStump()` 找到最佳单层决策树
    将最佳单层决策树加入单层决策树组
    计算alpha
    计算新的权重向量D
    更新累计类别估计值
    如果错误率等于0，则退出循环
    


In [6]:
classifierArray = adaboost.adaBoostTrainDS(dataArr=datMat, classLabels=classLabels, numIt=9)

total error:  0.2
total error:  0.2
total error:  0.0


In [7]:
classifierArray

[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565},
 {'dim': 0, 'thresh': 0.9, 'ineq': 'lt', 'alpha': 0.8958797346140273}]

算法在第3次迭代后错误率为0

测试AdaBoost算法，使用构建好的分类器 `classifierArray` 进行分类。

该分类器包含了三个单层决策树。

所以，这个分类器的预测值则为每个分类器的加权预测值之和

```python
aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix, classifierArr[i]['dim'],\
                                 classifierArr[i]['thresh'],\
                                 classifierArr[i]['ineq'])#call stump classify
        aggClassEst += classifierArr[i]['alpha']*classEst
        print(aggClassEst)
```

In [8]:
adaboost.adaClassify([0,0], classifierArray)

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])

`adaClassify()` 打印了分类器每次迭代预测值的累计值，其中 `[[-2.56198199]]` 为最终的预测值， `matrix([[-1.]])` 为分类标签


示例：在马疝病数据上应用AdaBoost



In [9]:
datArr, labelArr = adaboost.loadDataSet('horseColicTraining2.txt')
classifierArray = adaboost.adaBoostTrainDS(datArr, labelArr, 10)

total error:  0.2842809364548495
total error:  0.2842809364548495
total error:  0.24749163879598662
total error:  0.24749163879598662
total error:  0.25418060200668896
total error:  0.2408026755852843
total error:  0.2408026755852843
total error:  0.22073578595317725
total error:  0.24749163879598662
total error:  0.23076923076923078


In [13]:
testArr, testLabelArr = adaboost.loadDataSet('horseColicTest2.txt')
prediction10 = adaboost.adaClassify(testArr, classifierArray)
errArr = mat(ones((67,1)))
errArr[prediction10 != mat(testLabelArr).T].sum()

[[ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166

16.0